## QA Generation

Last time i did this it cost about $10 to create question:answer pairs, this time im gonna use local models to it using ollama.
This will iterate over the posts extracted from r/localllama and generate a QA dataset.

In [26]:
import tqdm
import ollama

In [29]:
# test
response = ollama.chat(model='mistral:latest', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
    'temperature': 0.01,
  },
])
print(response['message']['content'])

 The color of the sky appears blue due to a phenomenon called Rayleigh scattering. As sunlight reaches Earth's atmosphere, it interacts with molecules and particles in the air, such as nitrogen and oxygen. Blue light has a shorter wavelength and gets scattered more easily than other colors in the visible spectrum. As a result, when we look up at the sky, we predominantly see the blue light that has been scattered, giving the sky its characteristic blue hue during a clear day. However, at sunrise or sunset, the sky can take on various shades of red, orange, and purple as the sunlight interacts with more particles in the atmosphere and the angle of the sun's rays changes.


---

In [30]:
DATA_PATH = "./_output/new/localllama-new-17-02-2024.txt"
with open(DATA_PATH, "r") as file:
    data = file.read()

data_chunks = data.split("---\nPost ID:")

print(f"There are {len(data_chunks)} questions in total")
data_chunks[:3]

There are 975 questions in total


["Post ID: 1at0288\nTitle: Ok, which one of you was this? 🤣🤣🤣\nLink: https://redd.it/1at0288\nContent: \nReplies:\n- No, I don't think OpenAI would ever allow porn to be generated. I rather think that copies of Sora, recreated open source image generators will appear and fullfill this task. Porn is always one of the first use cases in any technologie that appeared and I don't think it'll take long for the industry to hop into this new tech. This is good for us as it further pushes open source AI technology for any use case.\n\n",
 ' 1aszy6f\nTitle: What are your favorite resources for evaluating text generation for stuff like readability, engagement (and other "soft" metrics)\nLink: https://redd.it/1aszy6f\nContent: Hi everyone, i\'m working on a thesis looking at different prompt engineering methods and trying to evaluate the quality of generated content for stuff like articles, newsletters = human read content. Most research focuses on stuff like factuality, reasoning but I\'m trying

In [31]:
response_chunks = []

print("Generating QA Pairs...")
for chunk in tqdm.tqdm(data_chunks):
    prompt = f"""
    ```
    {chunk}
    ```
    \n
    Your job is to look at this reddit post and to produce several question/answer pairs based on the content provided. 
    Look at the replies also and try to extract informative technical information. 
    Do not produce QA pairs for anything that is not in the provided text. 
    For longer posts (such as ones with a lot of information in the content or with many comments) produce a lot of QA pairs. 
    For posts with less content, produce fewer. Only include QA pairs with general useful information. 
    Do not include anything that could be considered personal information, opinion, or conversational text. 
    Only provide the QA pairs. Do NOT provide introductions or conclusions. Write your answer in this format:

    ```
    Q: What is the colour of the sky?
    A: The colour of the sky is blue.
    ---
    Q: How old is OpenAI? 
    A: OpenAI was founded in 2015, therefore it is 8 years old.
    ```
    """

    response_chunks.append(
        ollama.chat(model='mistral:latest', messages=[{
            'role': 'user',
            'content': prompt,
            'temperature': 0.2},
    ]))

print("Done!")

Generating QA Pairs...


  0%|          | 4/975 [01:06<4:27:20, 16.52s/it]


KeyboardInterrupt: 

In [32]:
prompt

'\n    ```\n     1asyo9m\nTitle: [LLaVa 1.6] How to write proper prompt that will give better image description without any redundant words?\nLink: https://redd.it/1asyo9m\nContent: Hello everyone.\n\nI\'ve managed to launch [LLaVa 1.6](https://github.com/haotian-liu/LLaVA)  13B vicuna version on my PC, and I\'ve figured out how to make streaming calls to it\'s API in order to caption images.\n\nMy prompt looks something like this:\n\n    A chat between a curious user and an artificial intelligence assistant. The assistant gives detailed answers. The assistant never refers to user and describes everything in third person view.\n    \n    USER: <image goes here>\n    Describe this scene in very detailed way. Styles are 3D and Cartoon.\n    ASSISTANT:\n\nI\'m calling API with **0.7 temperature.**\n\nModel returns really interesting results. However it starts with redundant words like this:\n\n    In the image, we see ...\n\nOr like these:\n\n    In this lively scene, we find ourselves ..